# 1. 3 Trade Balance

In [6]:
import pandas as pd
import requests
from io import BytesIO
import numpy as np
import os
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import kaggle

In [7]:
KAGGLE_USERNAME = "marpenalva"
KAGGLE_KEY = "ea42f53179cfa2f6eac83929293413f4"

kaggle.api.authenticate()

def download_file_from_kaggle(dataset, path):
    kaggle.api.dataset_download_files(dataset, path=path, unzip=True)

dataset = "appetukhov/international-trade-database"  # Dataset correcto
download_path = "data/"

if not os.path.exists(download_path):
    os.makedirs(download_path)

download_file_from_kaggle(dataset, download_path)


file_path = os.path.join(download_path, 'trade_1988_2021.csv')
df = pd.read_csv(file_path)

print(df.head())

,ReporterISO3,ReporterName,PartnerISO3,PartnerName,Year,TradeFlowName,TradeValue in 1000 USD
0,AFG,Afghanistan,SWE,Sweden,2017,Export,86.752
1,AFG,Afghanistan,JOR,Jordan,2018,Export,2796.481
2,AFG,Afghanistan,JOR,Jordan,2017,Export,3100.187
3,AFG,Afghanistan,ITA,Italy,2018,Export,279.918
4,AFG,Afghanistan,ITA,Italy,2017,Export,416.642
...,...,...,...,...,...,...,...
634504,ZWE,Zimbabwe,BRA,Brazil,2000,Export,1267.731
634505,ZWE,Zimbabwe,BOL,Bolivia,2000,Export,2.635
634506,ZWE,Zimbabwe,BMU,Bermuda,2002,Export,10.599
634507,ZWE,Zimbabwe,BLZ,Belize,2000,Export,17.772


In [8]:
df = df.drop(columns=['ReporterISO3', 'PartnerISO3'])
df = df.rename(columns={'ReporterName': 'Reporter', 'PartnerName': 'Partner', 'TradeFlowName': 'TradeFlow'})


In [9]:
palabra = 'World'
df = df[~df['Partner'].str.contains(palabra)]

In [10]:
def calculate_trade_balance(reporter, partner):
    imports = df[(df['Partner'] == reporter) & (df['Year'].between(2000, 2020))]
    exports = df[(df['Reporter'] == partner) & (df['Year'].between(2000, 2020))]
    
    imports_by_year = imports.groupby('Year')['TradeValue in 1000 USD'].sum().reset_index()
    exports_by_year = exports.groupby('Year')['TradeValue in 1000 USD'].sum().reset_index()
    
    imports_by_year.rename(columns={'TradeValue in 1000 USD': 'Imports'}, inplace=True)
    exports_by_year.rename(columns={'TradeValue in 1000 USD': 'Exports'}, inplace=True)
    
    trade_balance = pd.merge(imports_by_year, exports_by_year, on='Year', how='outer').fillna(0)
    trade_balance['Trade Balance'] = trade_balance['Exports'] - trade_balance['Imports']
    trade_balance['Country'] = reporter
    
    return trade_balance[['Year', 'Trade Balance', 'Country']]

china_balance = calculate_trade_balance('China', 'China')
us_balance = calculate_trade_balance('United States', 'United States')
germany_balance = calculate_trade_balance('Germany', 'Germany')

combined_balance = pd.concat([china_balance, us_balance, germany_balance])

fig = px.line(combined_balance, x='Year', y='Trade Balance', 
              color='Country',
              title='Trade Balance: China, United States and Germany (2000-2020)',
              markers=True)

pio.write_html(fig, file=f'graphs/balance/tradebalance.html', auto_open=False)